In [18]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import time
from sklearn.preprocessing import MinMaxScaler

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
!unzip /content/drive/MyDrive/dl-lab/dataset/stock.zip

Archive:  /content/drive/MyDrive/dl-lab/dataset/stock.zip
replace stock/stock_price.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: yes
  inflating: stock/stock_price.csv   


In [ ]:
data=pd.read_csv("stock/stock_price.csv")

In [ ]:
price = data[['Close']]
scaler = MinMaxScaler(feature_range=(-1, 1))
price['Close'] = scaler.fit_transform(price['Close'].values.reshape(-1,1))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
def split_data(stock, lookback):
    data_raw = stock.to_numpy() 
    data = []
    
    
    for index in range(len(data_raw) - lookback): 
        data.append(data_raw[index: index + lookback])
    
    data = np.array(data);
    test_set_size = int(np.round(0.2*data.shape[0]));
    train_set_size = data.shape[0] - (test_set_size);
    
    x_train = data[:train_set_size,:-1,:]
    y_train = data[:train_set_size,-1,:]
    
    x_test = data[train_set_size:,:-1]
    y_test = data[train_set_size:,-1,:]
    
    return [x_train, y_train, x_test, y_test]
lookback = 20 
x_train, y_train, x_test, y_test = split_data(price, lookback)

In [ ]:
x_train = torch.from_numpy(x_train).type(torch.Tensor)
x_test = torch.from_numpy(x_test).type(torch.Tensor)
y_train_lstm = torch.from_numpy(y_train).type(torch.Tensor)
y_test_lstm = torch.from_numpy(y_test).type(torch.Tensor)
y_train_gru = torch.from_numpy(y_train).type(torch.Tensor)
y_test_gru = torch.from_numpy(y_test).type(torch.Tensor)

In [ ]:
input_dim = 1
hidden_dim = 32
num_layers = 2
output_dim = 1
num_epochs = 100

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.fc(out[:, -1, :]) 
        return out

In [ ]:
model = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
criterion = torch.nn.MSELoss(reduction='mean')
optimiser = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:

hist = np.zeros(num_epochs)
start_time = time.time()
lstm = []
for t in range(num_epochs):
    y_train_pred = model(x_train)
    loss = criterion(y_train_pred, y_train_lstm)
    print("Epoch ", t, "MSE: ", loss.item())
    hist[t] = loss.item()
    optimiser.zero_grad()
    loss.backward()
    optimiser.step()
    
training_time = time.time()-start_time
print("Training time: {}".format(training_time))

Epoch  0 MSE:  0.26404842734336853
Epoch  1 MSE:  0.17929324507713318
Epoch  2 MSE:  0.12140277773141861
Epoch  3 MSE:  0.0915011316537857
Epoch  4 MSE:  0.13127167522907257
Epoch  5 MSE:  0.09386636316776276
Epoch  6 MSE:  0.07996277511119843
Epoch  7 MSE:  0.08237574994564056
Epoch  8 MSE:  0.0841292068362236
Epoch  9 MSE:  0.07850369065999985
Epoch  10 MSE:  0.06432095170021057
Epoch  11 MSE:  0.044905345886945724
Epoch  12 MSE:  0.034232351928949356
Epoch  13 MSE:  0.04289514943957329
Epoch  14 MSE:  0.018786940723657608
Epoch  15 MSE:  0.00518542667850852
Epoch  16 MSE:  0.017878932878375053
Epoch  17 MSE:  0.023922380059957504
Epoch  18 MSE:  0.010918916203081608
Epoch  19 MSE:  0.0045949979685246944
Epoch  20 MSE:  0.009652789682149887
Epoch  21 MSE:  0.009723394177854061
Epoch  22 MSE:  0.006440335884690285
Epoch  23 MSE:  0.006502971518784761
Epoch  24 MSE:  0.008578646928071976
Epoch  25 MSE:  0.009580004960298538
Epoch  26 MSE:  0.00824924185872078
Epoch  27 MSE:  0.00527661